## Import libraries

In [1]:
import json
import numpy as np
import random
import torch
import time
import pandas as pd
import gc

Making the notebook deterministic

In [2]:
def fix_random(seed: int) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


fix_random(seed=42)

In [3]:
data_path = "/kaggle/input/squad0"

This function loads a little SQuAD dataset for evaluation.

In [4]:
def load(path):
    data = []
    with open(f"{data_path}/squad_val_1k.jsonl", "r") as file:
          for line in file:
              try:
                  data.append(json.loads(line))
              except Exception as e:
                  print("json processing exception", e)
                  continue
    return data

In [5]:
data = load('squad_val_1k.jsonl')
print(data[0])

{'context': 'The Panthers beat the Seattle Seahawks in the divisional round, running up a 31–0 halftime lead and then holding off a furious second half comeback attempt to win 31–24, avenging their elimination from a year earlier. The Panthers then blew out the Arizona Cardinals in the NFC Championship Game, 49–15, racking up 487 yards and forcing seven turnovers.', 'prompt': 'How many yards did the Panthers get for the division championshipt game?', 'response': '487'}


This functions necessary for evaluating the models. More description below.

In [6]:
#This function generate the prompt. If we set the few_shot parameter to True, then it will generate
#a prompt, which uses few-shot prompting technique, otherwise not.
def generate_prompt(few_shot, question_with_context):
    questions = []
    if few_shot:
        three_shot_prompting = [
        {
            "context" : "In 1993, the FCC repealed the Financial Interest and Syndication Rules, once again allowing networks to hold interests in television production studios. That same year, Capital Cities/ABC purchased the French animation studio DIC Entertainment; it also signed an agreement with Time Warner Cable to carry its owned-and-operated television stations on the provider's systems in ABC O&O markets. By that year, ABC had a total viewership share of 23.63% of American households, just below the limit of 25% imposed by the FCC.",
            "question": "What French animation studio did ABC purchase in 1993?",
            "answer": "DIC Entertainment",
        },
        {
            "context" : "On April 12, 1961, Soviet cosmonaut Yuri Gagarin became the first person to fly in space, reinforcing American fears about being left behind in a technological competition with the Soviet Union. At a meeting of the US House Committee on Science and Astronautics one day after Gagarin's flight, many congressmen pledged their support for a crash program aimed at ensuring that America would catch up. Kennedy was circumspect in his response to the news, refusing to make a commitment on America's response to the Soviets.",
            "question": "How many days after Gagarin's flight did the US House Committee on Science and Astronautics meet?",
            "answer": "one day",
        },
        {
            "context" : "The customary law of Normandy was developed between the 10th and 13th centuries and survives today through the legal systems of Jersey and Guernsey in the Channel Islands. Norman customary law was transcribed in two customaries in Latin by two judges for use by them and their colleagues: These are the Très ancien coutumier (Very ancient customary), authored between 1200 and 1245; and the Grand coutumier de Normandie (Great customary of Normandy, originally Summa de legibus Normanniae in curia laïcali), authored between 1235 and 1245.",
            "question": "Where are Jersey and Guernsey?",
            "answer": "Channel Islands",
        }
        ]
        for item in question_with_context:
            prompt = f"You are a question answering bot. Your task is to answer the questions based on the appropriate contexts and your own knowledge. Your answers should contain only the most important things and should be as short as possible."
            prompt = f"{prompt}\n\n" + "\n\n".join([f"Question: {p['question']}\n\nContext: {p['context']}\n\nQuestion: {p['question']}\n\nAnswer: {p['answer']}" for p in three_shot_prompting])
            prompt = f"{prompt}\n\n{item}\n\nAnswer:"
            questions.append(prompt)
    else:
        for item in question_with_context:
            prompt = f"You are a question answering bot. Your task is to answer the questions based on the appropriate contexts and your own knowledge. Your answers should contain only the most important things and should be as short as possible."
            prompt = f"{prompt}\n\n{item}\n\nAnswer:"
            questions.append(prompt)
    return questions

#This function generate the text for the prompt.
def run_with_SQuAD(model, tokenizer, question, few_shot):
    prompt = generate_prompt(few_shot, question)
    
    inputs = tokenizer(prompt, padding=True, return_tensors="pt").to(0)
    model.to(0)

    #out = model.generate(inputs["input_ids"], max_new_tokens=128, do_sample=True, temperature=1.0, top_p=0.3, top_k=0, )
    
    out = model.generate(
        input_ids=inputs['input_ids'],
        max_new_tokens= + 128,
        attention_mask=inputs['attention_mask']
    )
    
    answers = []
    num_tokens = []
    
    decoded = tokenizer.batch_decode(out)

    for decodedText, promptText in zip(decoded, prompt):
        cleaned = decodedText.replace("<s>", "")
        num_tokens.append(len(cleaned) - len(promptText))
        cleaned = cleaned.replace(promptText, "")
        answer = cleaned.split("\n\n")[0].strip()
        answers.append(answer)

    return answers, num_tokens

#In this function we making the evaluation for every single datapoint. During the generation we
#are making a statistics about the generation.
def eval(data, model, tokenizer, output_file, few_shot, batch_size):
    numberOfTokens = 0
    timeOfStart = 0
    results = []
    total_qs = len(data)
    correct = 0
    i = 0

    for batch_start in range(0, total_qs, batch_size):
        batch_end = min(batch_start + batch_size, total_qs)
        batch_data = data[batch_start:batch_end] 

        # Prepare batch inputs
        questions = [item['prompt'] for item in batch_data]
        contexts = [item['context'] for item in batch_data]
        answers = [item['response'] for item in batch_data]
        inputs = [f"Question: {que}\n\nContext: {ctx}\n\nQuestion: {que}" for ctx, que in zip(contexts, questions)]
        
        # Generate outputs in a batch
        start_time = time.time()
        batch_guesses, batch_num_tokens = run_with_SQuAD(model, tokenizer, inputs, few_shot)
        end_time = time.time()
        timeOfStart += end_time - start_time

        # Process results 
        for guess, num_tokens, answer, question, context in zip(batch_guesses, batch_num_tokens, answers, questions, contexts):
            if guess and guess[-1] == '.' and answer[-1] != '.':
                guess = guess[:-1]
            numberOfTokens += num_tokens
            tkps = num_tokens / (end_time - start_time)
            is_correct = (answer.strip().lower() == guess.strip().lower())
            print(f"Question {i+1}/{total_qs}")
            print(f"Q: {question}")
            print(f"A: {answer}")
            print(f"?: {guess}")
            if is_correct:
                print(f"✅")
                correct += 1
            else:
                print(f"❌")
            print("="*80)
            result = {
                "idx": i,
                "question": question,
                "context": context,
                "answer": answer,
                "guess": guess,
                "is_correct": is_correct,
                "time": end_time - start_time,
                "num_tokens": num_tokens,
                "tokens_per_sec": tkps
            }
            results.append(result)
            i += 1
            
            if len(results) % 20 == 0:
                write_results(results, output_file)

    print(f"Accuracy: {correct / total_qs * 100}% -- {correct} correct and {total_qs - correct} incorrect")
    print(f"Number of tokens generated: {numberOfTokens} -- Time: {timeOfStart} -- Tokens-Per-Sec: {numberOfTokens / timeOfStart}")
    
#This function writes out the results.
def write_results(results, output_file):
    df = pd.DataFrame(results)
    df = df[["idx", "question", "context", "answer", "guess", "is_correct", "time", "num_tokens", "tokens_per_sec"]]
    print(f"Writing {output_file}")
    df.to_json(output_file, orient="records", lines=True)

## RWKV 169m model

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_rwkv169m = AutoModelForCausalLM.from_pretrained("RWKV/rwkv-4-world-169m", trust_remote_code=True).to(torch.float32)
tokenizer_rwkv169m = AutoTokenizer.from_pretrained("RWKV/rwkv-4-world-169m", trust_remote_code=True, padding_side='left', pad_token="<s>")

config.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/386M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

tokenization_rwkv5.py:   0%|          | 0.00/8.64k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/RWKV/rwkv-4-world-169m:
- tokenization_rwkv5.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


vocab.txt:   0%|          | 0.00/840k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/91.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPTNeoXTokenizerFast'. 
The class this function is called from is 'Rwkv5Tokenizer'.


In [8]:
eval(data[:100], model_rwkv169m, tokenizer_rwkv169m, "evalRWKV169M_batch.json", few_shot=False, batch_size=32)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question 1/100
Q: How many yards did the Panthers get for the division championshipt game?
A: 487
?: 
❌
Question 2/100
Q: What caused Jacksonville's tourism to become less desirable at the latter half of the 19th century?
A: yellow fever outbreaks
?: 
❌
Question 3/100
Q: What are pharmacists in the United Kingdom being increasingly paid for?
A: medicine use reviews
?: Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharmac Pharm

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question 33/100
Q: What type of materials inside the cabin were removed to help prevent more fire hazards in the future?
A: flammable cabin and space suit materials
?: The The The The The The the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
❌
Question 34/100
Q: How many nations contain "Amazonas" in their names?
A: States or departments in four nations contain "Amazonas" in their names.
?: Amazon Amazon Amazon Amazon Amazon Amazon Amazon Amazon Amazon Amazon Amazon Amazon Amazon Amazon Amazon Amazon Amazon Amazon Amazon Amazon Amazon Amazon Amazon Amazon 

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question 65/100
Q: What ABC action series went up against NBC's variety shows in Fall 1957?
A: Zorro
?: ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC ABC
❌
Question 66/100
Q: What was the significance of victory at Forth Niagara for British?
A: cut off the French frontier forts further to the west and south
?: The The The The The the the
❌
Question 67/100
Q: What is the Latin name for Black Death?
A: atra mors
?: 
❌
Question 68/100
Q: What are the phagocytes that are located in tissues in contact with the external environment called

In [9]:
eval(data[:100], model_rwkv169m, tokenizer_rwkv169m, "evalRWKV169M_few_shot_batch.json", few_shot=True, batch_size=32)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (1024). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Question 1/100
Q: How many yards did the Panthers get for the division championshipt game?
A: 487
?: The The The The The the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
❌
Question 2/100
Q: What caused Jacksonville's tourism to become less desirable at the latter half of the 19th century?
A: yellow fever outbreaks
?: Florida Florida Florida Florida Florida Florida Florida Florida Florida Florida Florida Florida Florida Florida Florida Florida Florida Florida Florida Florida Florida Florida Florida Florida Florida Florida Florida Florida Florida Florid

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question 33/100
Q: What type of materials inside the cabin were removed to help prevent more fire hazards in the future?
A: flammable cabin and space suit materials
?: The The The The The the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
❌
Question 34/100
Q: How many nations contain "Amazonas" in their names?
A: States or departments in four nations contain "Amazonas" in their names.
?: The The The the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the 

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question 65/100
Q: What ABC action series went up against NBC's variety shows in Fall 1957?
A: Zorro
?: NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC NBC
❌
Question 66/100
Q: What was the significance of victory at Forth Niagara for British?
A: cut off the French frontier forts further to the west and south
?: Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Battle Ba

Because when the batch size is not 1, then the model generate pointless answers, i set the batch size to 1 for this model.

In [10]:
eval(data[:100], model_rwkv169m, tokenizer_rwkv169m, "evalRWKV169M.json", few_shot=False, batch_size=1)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question 1/100
Q: How many yards did the Panthers get for the division championshipt game?
A: 487
?: The Panthers got for the division championshipt game, winning 31–14, winning 31–14. The Panthers got for the division championshipt game, winning 31–14, winning 31–14
❌


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question 2/100
Q: What caused Jacksonville's tourism to become less desirable at the latter half of the 19th century?
A: yellow fever outbreaks
?: Jacksonville's tourism was less desirable at the beginning of the 19th century due to its location on the St. Johns River and its proximity to the St. Johns River. However, the city's tourism industry continued to thrive during the 19th century. The city's tourism industry was also heavily influenced by the city's tourism industry. The city's tourism industry was heavily influenced by the city's tourism industry. The city's tourism industry was heavily influenced by the city's tourism industry. The city's tourism industry was also heavily influenced by the St. Johns River and Sailors Home. The city's tourism industry was also heavily influenced by the St
❌


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

Question 97/100
Q: How many days after Gagarin's flight did the US House Committee on Science and Astronautics meet?
A: one day
?: After Gagarin's flight, the US House Committee on Science and Astronautics met in Washington, DC on April 12, 1961. Kennedy was the first American to fly in space, and he was the first American to fly in space. Kennedy was the first American to fly in space, and he was the first American to fly in space. Kennedy was the first American to fly in space, and he was the first American to fly in space
❌


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question 98/100
Q: How can function problems typically be restated?
A: decision problems
?: Function problems can be restated as a set of triples such that the set of all possible values is a set of all possible values. For example, the set of all possible values is a set of all possible values. The set of all possible values is a set of all possible values. The set of all possible values is a set of all possible values. The set of all possible values is a set of all possible values. The set of all possible values is a set of all possible values. The set of all possible values is a set of all possible values. The set of all possible values is a set of all possible values
❌


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question 99/100
Q: What does Fortiter Defendit Triumphans mean?
A: Triumphing by a brave defence
?: Fortiter Defendit Triumphans is a medieval English word meaning "to conquer". It is a medieval English word that means "to conquer". It is also used in the English language as a metaphor for the British army's defeat in the Battle of Newburn
❌
Question 100/100
Q: Recent studies believe  that ctenophores are the sister lineage to what?
A: all other animals
?: The most likely explanation for Ctenophores is that they are the sister lineage to the ctenophore family, which includes all the other ctenophore lineages. This means that Ctenophores are the sister lineage to all other ctenophores
❌
Writing evalRWKV169M.json
Accuracy: 0.0% -- 0 correct and 100 incorrect
Number of tokens generated: 55557 -- Time: 201.20423221588135 -- Tokens-Per-Sec: 276.1224224169913


In [11]:
eval(data[:100], model_rwkv169m, tokenizer_rwkv169m, "evalRWKV169M_Few_shot.json", few_shot=True, batch_size=1)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question 1/100
Q: How many yards did the Panthers get for the division championshipt game?
A: 487
?: The Panthers got for the Panthers a total of 12 yards in the division championshipt game, defeating the Seattle Seahawks in the division championshipt game
❌


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question 2/100
Q: What caused Jacksonville's tourism to become less desirable at the latter half of the 19th century?
A: yellow fever outbreaks
?: Jacksonville's tourism was less desirable at the beginning of the 19th century. The city's tourism industry was heavily dependent on tourism, and the city's tourism industry was heavily dependent on tourism. The city's tourism industry was heavily dependent on tourism, and the city's tourism industry was heavily dependent on tourism. The city's tourism industry was heavily dependent on tourism, and the city's tourism industry was heavily dependent on tourism. The city's tourism industry was heavily dependent on tourism, and the city's tourism industry was heavily dependent on tourism. The city's tourism industry was heavily dependent on tourism, and the city's tourism industry was heavily dependent on
❌


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

Question 97/100
Q: How many days after Gagarin's flight did the US House Committee on Science and Astronautics meet?
A: one day
?: On April 12, 1961, Kennedy met with the US House Committee on Science and Astronautics in Washington, D.C. to discuss the possibility of a meeting in Paris. Kennedy was not present, but the US House Committee on Science and Astronautics met in Paris on April 12, 1961. Kennedy was the first American to meet in person in Paris, and he was the first to meet in person
❌


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question 98/100
Q: How can function problems typically be restated?
A: decision problems
?: Function problems can be restated by using the following rules:
❌


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question 99/100
Q: What does Fortiter Defendit Triumphans mean?
A: Triumphing by a brave defence
?: Fortiter Defendit Triumphans is a medieval English word for "fortification". It is a medieval English word that means "fortification". It is a medieval English word that means "fortification". The word "fortition" is a derivative of the Latin fortis, meaning "fortification". The word "fortition" is a Latin word that means "fortification". The word "fortition" is a Latin word that means "fortification". The word "fortition" is a Latin word that means "fortification". The word "fortition" is a Latin word that means "fortification". The word
❌
Question 100/100
Q: Recent studies believe  that ctenophores are the sister lineage to what?
A: all other animals
?: Ctenophores are the sister lineage to all other animal phyla, including all other animal phyla. They are the closest living relatives of ctenophores, and they are the only animal phyla that have been studied in detail. Ctenophores are t

In [12]:
model_rwkv169m = None
tokenizer_rwkv169m = None

gc.collect()

36

## RWKV 3B model

In [13]:
!pip install flash-rwkv

In [14]:
model_rwkv_3b = AutoModelForCausalLM.from_pretrained("RWKV/rwkv-5-world-3b", trust_remote_code=True).to(torch.float32)
tokenizer_rwkv_3b = AutoTokenizer.from_pretrained("RWKV/rwkv-5-world-3b", trust_remote_code=True, padding_side='left', pad_token="<s>")

config.json:   0%|          | 0.00/584 [00:00<?, ?B/s]

configuration_rwkv5.py:   0%|          | 0.00/5.08k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/RWKV/rwkv-5-world-3b:
- configuration_rwkv5.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_rwkv5.py:   0%|          | 0.00/32.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/RWKV/rwkv-5-world-3b:
- modeling_rwkv5.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Using /root/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/py310_cu121/flash_rwkv_5...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py310_cu121/flash_rwkv_5/build.ninja...
Building extension module flash_rwkv_5...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


...

tokenization_rwkv5.py:   0%|          | 0.00/8.60k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/RWKV/rwkv-5-world-3b:
- tokenization_rwkv5.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


vocab.txt:   0%|          | 0.00/840k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/15.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [15]:
eval(data[:100], model_rwkv_3b, tokenizer_rwkv_3b, "evalRWKV3b.json", few_shot=False, batch_size=1)

Question 1/100
Q: How many yards did the Panthers get for the division championshipt game?
A: 487
?: The Panthers got for the division championshipt game
❌
Question 2/100
Q: What caused Jacksonville's tourism to become less desirable at the latter half of the 19th century?
A: yellow fever outbreaks
?: Yellow fever outbreaks
✅
Question 3/100
Q: What are pharmacists in the United Kingdom being increasingly paid for?
A: medicine use reviews
?: Pharmacists in the United Kingdom are being increasingly paid for by the government for medicine use reviews
❌
Question 4/100
Q: Geoglyphs dating to what period were found in deforested land along the Amazon River?
A: AD 0–1250
?: Geoglyphs dating to what period were found in deforested land along the Amazon River?
❌
Question 5/100
Q: Who does a gender pay gap tend to favor?
A: males in the labor market
?: A gender pay gap tends to favor males in the labor market
❌
Question 6/100
Q: What is the annual construction industry revenue in 2014?
A: $960 b

In [16]:
eval(data[:100], model_rwkv_3b, tokenizer_rwkv_3b, "evalRWKV3b_few_shot.json", few_shot=True, batch_size=1)

Question 1/100
Q: How many yards did the Panthers get for the division championshipt game?
A: 487
?: 49
❌
Question 2/100
Q: What caused Jacksonville's tourism to become less desirable at the latter half of the 19th century?
A: yellow fever outbreaks
?: Yellow fever outbreaks
✅
Question 3/100
Q: What are pharmacists in the United Kingdom being increasingly paid for?
A: medicine use reviews
?: Pharmacists in the United Kingdom are being increasingly paid for their work
❌
Question 4/100
Q: Geoglyphs dating to what period were found in deforested land along the Amazon River?
A: AD 0–1250
?: Pre-Columbian
❌
Question 5/100
Q: Who does a gender pay gap tend to favor?
A: males in the labor market
?: women
❌
Question 6/100
Q: What is the annual construction industry revenue in 2014?
A: $960 billion
?: $960 billion
✅
Question 7/100
Q: Which sculpture by Michelangelo has a full-size replica in the Cast Courts?
A: David
?: Michelangelo's David
❌
Question 8/100
Q: Richard Allen and Absalom Jones we

In [17]:
eval(data[:100], model_rwkv_3b, tokenizer_rwkv_3b, "evalRWKV3b_batch.json", few_shot=False, batch_size=8)

Question 1/100
Q: How many yards did the Panthers get for the division championshipt game?
A: 487
?: The Panthers got for the divisional round, 49–15, racking up 487 yards and forcing seven turnovers
❌
Question 2/100
Q: What caused Jacksonville's tourism to become less desirable at the latter half of the 19th century?
A: yellow fever outbreaks
?: The city's tourism declined because of the following factors:
❌
Question 3/100
Q: What are pharmacists in the United Kingdom being increasingly paid for?
A: medicine use reviews
?: Pharmacists in the United Kingdom are being increasingly paid for by the government for medicine use reviews
❌
Question 4/100
Q: Geoglyphs dating to what period were found in deforested land along the Amazon River?
A: AD 0–1250
?: The first European to travel the length of the Amazon River was Francisco de Orellana in 1542. The BBC's Unnatural Histories presented evidence that Orellana, rather than exaggerating his claims as previously thought, was correct in his ob

In [18]:
eval(data[:100], model_rwkv_3b, tokenizer_rwkv_3b, "evalRWKV3b_Few_shot_batch.json", few_shot=True, batch_size=8)

Question 1/100
Q: How many yards did the Panthers get for the division championshipt game?
A: 487
?: 31
❌
Question 2/100
Q: What caused Jacksonville's tourism to become less desirable at the latter half of the 19th century?
A: yellow fever outbreaks
?: Yellow fever outbreaks
✅
Question 3/100
Q: What are pharmacists in the United Kingdom being increasingly paid for?
A: medicine use reviews
?: Pharmacists in the United Kingdom are being increasingly paid for their work
❌
Question 4/100
Q: Geoglyphs dating to what period were found in deforested land along the Amazon River?
A: AD 0–1250
?: Deforested land along the Amazon River
❌
Question 5/100
Q: Who does a gender pay gap tend to favor?
A: males in the labor market
?: Women
❌
Question 6/100
Q: What is the annual construction industry revenue in 2014?
A: $960 billion
?: $680 billion
❌
Question 7/100
Q: Which sculpture by Michelangelo has a full-size replica in the Cast Courts?
A: David
?: Michelangelo's David
❌
Question 8/100
Q: Richard A

In [19]:
tokenizer_rwkv_3b = None
model_rwkv_3b = None

gc.collect()

8